In [1]:
import numpy as np
from torchvision import datasets, transforms    
import torch
from torch.utils.data import DataLoader, random_split

In [2]:
from model import SimpleCNN
model = SimpleCNN()

In [3]:
from loader import load
train_loader, val_loader, test_loader = load()

In [4]:
from trainer import train as train_model
from optimizer import RandomOptimizer 

In [5]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")

model.to(device)

criterion = torch.nn.MSELoss()
optimizer = RandomOptimizer(model.parameters()) 

model = train_model(model, train_loader, val_loader, optimizer, criterion, device, 5)

Input size:  64


/opt/homebrew/Caskroom/miniforge/base/envs/optml/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (10) must match the size of tensor b (64) at non-singleton dimension 1